In [1]:
import pandas as pd
import numpy as np
import warnings
import glob
import re
import datetime
import calendar

#### Inputs para Ejecución del Proceso

In [2]:
while True:
    try:
        mes_cierre = int(input("Indique el mes de cierre (MM): "))
        if mes_cierre < 1 or mes_cierre > 12:
            raise ValueError("El mes debe estar entre 1 y 12.")
        break
    except ValueError as e:
        print(f"Entrada inválida: {e}. Por favor, ingrese un número válido para el mes.")

while True:
    try:
        año_cierre = int(input("Indique el año de cierre (AAAA): "))
        if año_cierre < 1000 or año_cierre > 9999:
            raise ValueError("El año debe tener cuatro dígitos.")
        break
    except ValueError as e:
        print(f"Entrada inválida: {e}. Por favor, ingrese un número válido para el año.")

# Funcion para generar la fecha con el ultimo dia del mes de cierre
def ultimo_dia_del_mes(mes_cierre, año_cierre):
    ultimo_dia = calendar.monthrange(año_cierre, mes_cierre)[1]
    fecha = datetime.date(año_cierre, mes_cierre, ultimo_dia)
    return fecha.strftime("%d%m%Y")

ultimo_dia_del_mes = ultimo_dia_del_mes(mes_cierre, año_cierre)

print(f"La fecha de cierre es: {ultimo_dia_del_mes}")

30062024


#### Definición de Párametros Iniciales

In [3]:
# Rango de tolerancia para eliminación de registros con valor muy pequeño
tolerancia_eliminacon_cero = 0.5
desde_eliminacon_cero = -tolerancia_eliminacon_cero
hasta_eliminacon_cero = tolerancia_eliminacon_cero

# Rango de tolerancia para eliminación de pólizas cerradas
tolerancia_filtros_iniciales = 10
desde = -tolerancia_filtros_iniciales
hasta = tolerancia_filtros_iniciales

# Rango de tolerancia para ajustar pólizas en plantilla ajustes financieros
tolerancia_ajustes_financieros_desde = 10
tolerancia_ajustes_financieros_hasta = 1000
desde_positivo = tolerancia_ajustes_financieros_desde
desde_negativo = -tolerancia_ajustes_financieros_desde
hasta_positivo = tolerancia_ajustes_financieros_hasta
hasta_negativo = -tolerancia_ajustes_financieros_hasta

valor_no_ajuste = 6000
desde_no_ajuste = -valor_no_ajuste
hasta_no_ajuste = valor_no_ajuste

# Rango de tolerancia para ajustar pólizas en dólares
tolerancia_porc_pendientes = 0.1
desde_porc_pendientes = -tolerancia_porc_pendientes
hasta_porc_pendientes = tolerancia_porc_pendientes

tipos_fuente = pd.read_excel('01. Input/Tipos Texto.xlsx')

# Lista de frases para identificar las provisiones
provision = tipos_fuente.loc[tipos_fuente['TIPO'] == "Provision"]['TEXTO'].tolist()

# Lista de frases para identificar los pagos
pago = tipos_fuente.loc[tipos_fuente['TIPO'] == "Pago"]['TEXTO'].tolist()

# Lista de frases para identificar los ajustes de provision
ajuste_provision = tipos_fuente.loc[tipos_fuente['TIPO'] == "Ajuste_Provision"]['TEXTO'].tolist()

# Lista de frases para identificar los ajustes de pago
ajuste_pago = tipos_fuente.loc[tipos_fuente['TIPO'] == "Ajuste_Pago"]['TEXTO'].tolist()

# Lista CDTIPO_FUENTE completa
cdtipo_fuente = tipos_fuente['TEXTO'].tolist()

#### Importación de Archivos

In [4]:
# Archivos CSV a concatenar
all_files_generales = glob.glob('01. Input/01. Generales/*.txt')

# Esquema de datos
cabecera ={"CDSOCIEDAD": "object",
            "CDRAMO": "object",
            "NMPOLIZA": "object",
            "NMRECIBO": "object",
            "CDCONCEPTO": "object",
            "CDCOASEGURO": "object",
            "CDAGENTE": "object",
            "CDDELEGACION": "object",
            "PTPRIMA": "float",
            "PTCOMISION": "float",
            "POCOMISION": "float",
            "CDTIPO_FUENTE": "object",
            "FEDOCUMENTO": "object",
            "FEREGISTRO": "object",
            "FEINI_VIGENCIA": "object",
            "FEFIN_FIGENCIA": "object",
            "CDLIBRO":"object",
            "VACIO":"object"}

# Lista para almacenar los DataFrames de los archivos CSV
dataframes_generales= []

# Cargar cada archivo CSV en un DataFrame y agregarlo a la lista
for file in all_files_generales:
    warnings.filterwarnings('ignore')
    df = pd.read_table(file, sep=';', header=None, names=list(cabecera.keys()), dtype=cabecera)
    dataframes_generales.append(df)
    print(f'Archivo {file} cargado. Tiene {df.shape[0]} filas')

# Archivo principal Generales
generales = pd.concat(dataframes_generales, ignore_index=True)
generales = generales.drop('VACIO', axis=1)
generales = generales.loc[generales['PTCOMISION'].notnull()]
generales = generales.loc[~(generales['PTCOMISION'].between(desde_eliminacon_cero, hasta_eliminacon_cero, inclusive='both'))]
generales = generales.fillna("")

Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_6106901.txt cargado. Tiene 799999 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069010.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069011.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069012.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069013.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069014.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069015.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069016.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. Generales\libro_auxiliar_prov_comisiones_gen_61069017.txt cargado. Tiene 800000 filas
Archivo 01. Input/01. General

In [5]:
generales['PTCOMISION'].sum()

np.float64(35197816869.1301)

In [6]:
## Insumos varios
dolares = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='Dolares', dtype=str)
retirados = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='Retirados', dtype={'CDAGENTE':'str'})
codigos_directos = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='Codigos directos', dtype=str)
asesores_formacion = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='FyD', dtype={'CDAGENTE':'str'})

# Transformación Insumos
dolares['DOLARES'] = "X"
retirados['RETIRADOS'] = "X"
codigos_directos['CDDIR'] = "X"
asesores_formacion['FYD'] = "X"

# Ajustar NMPOLIZA en el archivo dolares para que tenga como mínimo 12 dígitos
dolares['NMPOLIZA'] = np.select(
    [dolares['NMPOLIZA'].str.len() < 12],
    [dolares['NMPOLIZA'].str.rjust(12, "0")],
    default=dolares['NMPOLIZA']
)

# Seleccionar columnas necesarias en el archivo retirados
retirados = retirados[['CDAGENTE', 'RETIRADOS', 'FEBAJA']]

#### Validación CDTIPO_FUENTE

In [7]:
cdtipo_fuente_validar = generales['CDTIPO_FUENTE'].unique()

# Crear una lista de elementos que están en cdtipo_fuente_validar pero no en cdtipo_fuente
elementos_no_encontrados = [elemento for elemento in cdtipo_fuente_validar if elemento not in cdtipo_fuente]

# Verificar si hay elementos no encontrados
if elementos_no_encontrados:
    print("Los siguientes elementos no se encuentran en cdtipo_fuente:")
    for elemento in elementos_no_encontrados:
        print(elemento)
else:
    print("Todos los elementos de cdtipo_fuente_validar se encuentran en cdtipo_fuente.")


Los siguientes elementos no se encuentran en cdtipo_fuente:
RECLAS SOBRCOM CORE  04-2020


#### Transformaciones Iniciales

In [41]:
# Ajustar NMPOLIZA para que tenga como mínimo 12 dígitos
generales['NMPOLIZA'] = np.select(
    [generales['NMPOLIZA'].str.len() < 12],
    [generales['NMPOLIZA'].str.rjust(12, "0")],
    default=generales['NMPOLIZA']
)

# Ajustar CDRAMO para que tenga como mínimo 3 dígitos
generales['CDRAMO'] = np.select(
    [generales['CDRAMO'].str.len() < 3],
    [generales['CDRAMO'].str.rjust(3, "0")],
    default=generales['CDRAMO']
)

# Quitar letras y caracteres especiales de NMRECIBO y convertir a número
generales['NMRECIBO'] = generales['NMRECIBO'].apply(lambda x: re.sub(r'[^0-9]', '', x))
generales['NMRECIBO'] = pd.to_numeric(generales['NMRECIBO'], errors='coerce')

# Agregar tipo M a CDCOASEGURO
generales['CDCOASEGURO'] = np.select(
    [generales['CDTIPO_FUENTE'].isin(ajuste_pago),
     (generales['CDCOASEGURO'] == "") & (generales['CDTIPO_FUENTE'].isin(pago))], 
    ["M",
     "M"],
    default=generales['CDCOASEGURO']
)

#### Depuración de Tablas

In [42]:
## Eliminación de Pólizas con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza para determinar su valor y marcar las que tienen valor 0
generales_depurada1 = generales.groupby('NMPOLIZA')['PTCOMISION'].sum().reset_index()
generales_depurada1['ELIMINAR?'] = np.select(
    [generales_depurada1['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
generales_depurada1 = generales_depurada1.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas con valor 0
generales = generales.merge(generales_depurada1, on='NMPOLIZA', how='left')
generales = generales.loc[generales['ELIMINAR?'] != "X"]
generales = generales.drop('ELIMINAR?', axis=1)

In [43]:
## Eliminación de Pólizas/Recibo con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza/recibo para determinar su valor y marcar las que tienen valor 0
generales_depurada2 = generales[(generales['NMRECIBO'] != "99999999") | (generales['NMRECIBO'] != "9999999") | (generales['NMRECIBO'] != "999999") | (generales['CDCOASEGURO']) != "M"]
generales_depurada2 = generales_depurada2.groupby(['NMPOLIZA','NMRECIBO'])['PTCOMISION'].sum().reset_index()
generales_depurada2['ELIMINAR?'] = np.select(
    [generales_depurada2['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
generales_depurada2 = generales_depurada2.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/recibo con valor 0
generales = generales.merge(generales_depurada2, on=['NMPOLIZA','NMRECIBO'], how='left')
generales = generales.loc[generales['ELIMINAR?'] != "X"]
generales = generales.drop('ELIMINAR?', axis=1)

In [44]:
## Eliminación de Pólizas/Agente con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza/agente para determinar su valor y marcar las que tienen valor 0
generales_depurada3 = generales[(generales['NMRECIBO'] != "99999999") | (generales['NMRECIBO'] != "9999999") | (generales['NMRECIBO'] != "999999") | (generales['CDCOASEGURO']) != "M"]
generales_depurada3 = generales_depurada3.groupby(['NMPOLIZA','CDAGENTE'])['PTCOMISION'].sum().reset_index()
generales_depurada3['ELIMINAR?'] = np.select(
    [generales_depurada3['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
generales_depurada3 = generales_depurada3.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/agente con valor 0
generales = generales.merge(generales_depurada3, on=['NMPOLIZA','CDAGENTE'], how='left')
generales = generales.loc[generales['ELIMINAR?'] != "X"]
generales = generales.drop('ELIMINAR?', axis=1)

In [45]:
## Eliminación de Pólizas por código coaseguro con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por código coaseguro para determinar su valor y marcar las que tienen valor 0
generales_depurada4 = generales.loc[generales['CDCOASEGURO'] == "M"]
generales_depurada4 = generales_depurada4.groupby(['NMPOLIZA','CDCOASEGURO','FEREGISTRO'])['PTCOMISION'].sum().reset_index()
generales_depurada4['ELIMINAR?'] = np.select(
    [generales_depurada4['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
generales_depurada4 = generales_depurada4.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/agente con valor 0
generales = generales.merge(generales_depurada4, on=['NMPOLIZA','CDCOASEGURO','FEREGISTRO'], how='left')
generales = generales.loc[generales['ELIMINAR?'] != "X"]
generales = generales.drop('ELIMINAR?', axis=1)

In [46]:
## Eliminación de Pólizas/Recibo con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza/recibo para determinar su valor y marcar las que tienen valor 0
generales_depurada5 = generales[(generales['NMRECIBO'] != "99999999") | (generales['NMRECIBO'] != "9999999") | (generales['NMRECIBO'] != "999999") | (generales['CDCOASEGURO']) != "M"]
generales_depurada5 = generales_depurada5.groupby(['NMPOLIZA','NMRECIBO'])['PTCOMISION'].sum().reset_index()
generales_depurada5['ELIMINAR?'] = np.select(
    [generales_depurada5['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
generales_depurada5 = generales_depurada5.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/recibo con valor 0
generales = generales.merge(generales_depurada5, on=['NMPOLIZA','NMRECIBO'], how='left')
generales = generales.loc[generales['ELIMINAR?'] != "X"]
generales = generales.drop('ELIMINAR?', axis=1)

In [47]:
## Eliminación de Pólizas con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza para determinar su valor y marcar las que tienen valor 0
generales_depurada6 = generales.groupby('NMPOLIZA')['PTCOMISION'].sum().reset_index()
generales_depurada6['ELIMINAR?'] = np.select(
    [generales_depurada6['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
generales_depurada6 = generales_depurada6.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas con valor 0
generales = generales.merge(generales_depurada6, on='NMPOLIZA', how='left')
generales = generales.loc[generales['ELIMINAR?'] != "X"]
generales = generales.drop('ELIMINAR?', axis=1)

#### Identificación para Ajustes Financieros

In [48]:
## Identificación de registros para ajustes financieros por póliza

# Agrupar por póliza para determinar su valor y marcar las que tienen valor apto para ajuste
generales_ajustes1_temp = generales.groupby(['NMPOLIZA'])['PTCOMISION'].sum().reset_index()
generales_ajustes1_temp['AJUSTAR?'] = np.select(
    [((generales_ajustes1_temp['PTCOMISION'].between(hasta_negativo, desde_negativo, inclusive='both')) | 
     (generales_ajustes1_temp['PTCOMISION'].between(desde_positivo, hasta_positivo, inclusive='both')))],
    ["Ajustado Póliza"],
    default=""
)
generales_ajustes1_temp = generales_ajustes1_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
generales_ajustes1 = generales.merge(generales_ajustes1_temp, on=['NMPOLIZA'], how='left')

# Identificar cuales registros que se marcaron como Ajuste no aplican realmente
generales_ajustes1['REV_AJUSTE'] = np.select(
    [(generales_ajustes1['AJUSTAR?'] == "Ajustado Póliza") & (generales_ajustes1['CDTIPO_FUENTE'].isin(provision)) & (generales_ajustes1['PTCOMISION'].between(desde_no_ajuste, hasta_no_ajuste, inclusive='both'))], 
    ["NO"],
    default="SI"
)
generales_rev_ajustes1 = generales_ajustes1.loc[generales_ajustes1['REV_AJUSTE'] == "NO"]
generales_rev_ajustes1 = generales_rev_ajustes1.groupby(['NMPOLIZA', 'REV_AJUSTE'])['REV_AJUSTE'].count().reset_index(name='CUENTA_REV_AJUSTE')
generales_rev_ajustes1 = generales_rev_ajustes1.drop('CUENTA_REV_AJUSTE', axis=1)
generales_ajustes1 = generales_ajustes1.drop('REV_AJUSTE', axis=1)
generales_ajustes1 = generales_ajustes1.merge(generales_rev_ajustes1, on=['NMPOLIZA'], how='left')

# Quitar marca de ajuste a las pólizas que tienen registros con valores "pequeños"
generales_ajustes1['AJUSTAR?'] = np.select(
    [(generales_ajustes1['AJUSTAR?'] == "Ajustado Póliza") & (generales_ajustes1['REV_AJUSTE'] == "NO")],
    [""],
    default=generales_ajustes1['AJUSTAR?']
)
generales_ajustes1 = generales_ajustes1.drop('REV_AJUSTE', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
generales_ajustados1 = generales_ajustes1.loc[generales_ajustes1['AJUSTAR?'] == "Ajustado Póliza"]
generales_revision1 = generales_ajustes1.loc[generales_ajustes1['AJUSTAR?'] != "Ajustado Póliza"]

In [49]:
## Identificación de registros para ajustes financieros por póliza/recibo

generales_revision1 = generales_revision1.drop('AJUSTAR?', axis=1)

# Agrupar por póliza/recibo para determinar su valor y marcar las que tienen valor apto para ajuste
generales_ajustes2_temp = generales_revision1.groupby(['NMPOLIZA', 'NMRECIBO'])['PTCOMISION'].sum().reset_index()
generales_ajustes2_temp['AJUSTAR?'] = np.select(
    [((generales_ajustes2_temp['PTCOMISION'].between(hasta_negativo, desde_negativo, inclusive='both')) | 
     (generales_ajustes2_temp['PTCOMISION'].between(desde_positivo, hasta_positivo, inclusive='both')))],
    ["Ajustado Póliza/Recibo"],
    default=""
)
generales_ajustes2_temp = generales_ajustes2_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
generales_ajustes2 = generales_revision1.merge(generales_ajustes2_temp, on=['NMPOLIZA', 'NMRECIBO'], how='left')

# Identificar cuales registros que se marcaron como Ajuste no aplican realmente
generales_ajustes2['REV_AJUSTE'] = np.select(
    [(generales_ajustes2['AJUSTAR?'] == "Ajustado Póliza/Recibo") & (generales_ajustes2['PTCOMISION'].between(desde_no_ajuste, hasta_no_ajuste, inclusive='both'))], 
    ["NO"],
    default="SI"
)
generales_rev_ajustes2 = generales_ajustes2.loc[generales_ajustes2['REV_AJUSTE'] == "NO"]
generales_rev_ajustes2 = generales_rev_ajustes2.groupby(['NMPOLIZA', 'NMRECIBO', 'REV_AJUSTE'])['REV_AJUSTE'].count().reset_index(name='CUENTA_REV_AJUSTE')
generales_rev_ajustes2 = generales_rev_ajustes2.drop('CUENTA_REV_AJUSTE', axis=1)
generales_ajustes2 = generales_ajustes2.drop('REV_AJUSTE', axis=1)
generales_ajustes2 = generales_ajustes2.merge(generales_rev_ajustes2, on=['NMPOLIZA', 'NMRECIBO'], how='left')

# Quitar marca de ajuste a las pólizas que tienen registros con valores "pequeños"
generales_ajustes2['AJUSTAR?'] = np.select(
    [(generales_ajustes2['AJUSTAR?'] == "Ajustado Póliza/Recibo") & (generales_ajustes2['REV_AJUSTE'] == "NO")],
    [""],
    default=generales_ajustes2['AJUSTAR?']
)
generales_ajustes2 = generales_ajustes2.drop('REV_AJUSTE', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
generales_ajustados2 = generales_ajustes2.loc[generales_ajustes2['AJUSTAR?'] == "Ajustado Póliza/Recibo"]
generales_revision2 = generales_ajustes2.loc[generales_ajustes2['AJUSTAR?'] != "Ajustado Póliza/Recibo"]

In [50]:
## Identificación de registros para ajustes financieros por póliza/agente

generales_revision2 = generales_revision2.drop('AJUSTAR?', axis=1)

# Agrupar por póliza/agente para determinar su valor y marcar las que tienen valor apto para ajuste
generales_ajustes3_temp = generales_revision2.groupby(['NMPOLIZA', 'CDAGENTE'])['PTCOMISION'].sum().reset_index()
generales_ajustes3_temp['AJUSTAR?'] = np.select(
    [((generales_ajustes3_temp['PTCOMISION'].between(hasta_negativo, desde_negativo, inclusive='both')) | 
     (generales_ajustes3_temp['PTCOMISION'].between(desde_positivo, hasta_positivo, inclusive='both')))],
    ["Ajustado Póliza/Agente"],
    default=""
)
generales_ajustes3_temp = generales_ajustes3_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
generales_ajustes3 = generales_revision2.merge(generales_ajustes3_temp, on=['NMPOLIZA', 'CDAGENTE'], how='left')

# Identificar cuales registros que se marcaron como Ajuste no aplican realmente
generales_ajustes3['REV_AJUSTE'] = np.select(
    [(generales_ajustes3['AJUSTAR?'] == "Ajustado Póliza/Agente") & (generales_ajustes3['PTCOMISION'].between(desde_no_ajuste, hasta_no_ajuste, inclusive='both'))], 
    ["NO"],
    default="SI"
)
generales_rev_ajustes3 = generales_ajustes3.loc[generales_ajustes3['REV_AJUSTE'] == "NO"]
generales_rev_ajustes3 = generales_rev_ajustes3.groupby(['NMPOLIZA', 'CDAGENTE', 'REV_AJUSTE'])['REV_AJUSTE'].count().reset_index(name='CUENTA_REV_AJUSTE')
generales_rev_ajustes3 = generales_rev_ajustes3.drop('CUENTA_REV_AJUSTE', axis=1)
generales_ajustes3 = generales_ajustes3.drop('REV_AJUSTE', axis=1)
generales_ajustes3 = generales_ajustes3.merge(generales_rev_ajustes3, on=['NMPOLIZA', 'CDAGENTE'], how='left')

# Quitar marca de ajuste a las pólizas que tienen registros con valores "pequeños"
generales_ajustes3['AJUSTAR?'] = np.select(
    [(generales_ajustes3['AJUSTAR?'] == "Ajustado Póliza/Agente") & (generales_ajustes3['REV_AJUSTE'] == "NO")],
    [""],
    default=generales_ajustes3['AJUSTAR?']
)
generales_ajustes3 = generales_ajustes3.drop('REV_AJUSTE', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
generales_ajustados3 = generales_ajustes3.loc[generales_ajustes3['AJUSTAR?'] == "Ajustado Póliza/Agente"]
generales_revision3 = generales_ajustes3.loc[generales_ajustes3['AJUSTAR?'] != "Ajustado Póliza/Agente"]

In [51]:
## Identificación de registros para ajustes por recibo duplicado

# Identificar los recibos que se encuentran en pólizas distintas
recibos_duplicados = generales_revision3.groupby(['NMPOLIZA', 'NMRECIBO'])['PTCOMISION'].sum().reset_index()
recibos_duplicados['NMRECIBO_DUPLICADO'] = recibos_duplicados.duplicated(subset=['NMRECIBO'], keep=False)
recibos_duplicados = recibos_duplicados.sort_values(by=['NMRECIBO'], ascending=[True])
recibos_duplicados['OBSERVACION'] = np.select(
    [recibos_duplicados['NMRECIBO_DUPLICADO'] == True],
    ["Recibo Duplicado"],
    default=""
)
recibos_duplicados = recibos_duplicados.loc[recibos_duplicados['OBSERVACION'] == "Recibo Duplicado"]

# Agrupar por recibos duplicados para determinar su valor y marcar las que tienen valor apto para ajuste (Tolerancia definida en parametros iniciales)
generales_revision3 = generales_revision3.drop('AJUSTAR?', axis=1)

generales_ajustes4_temp = recibos_duplicados.groupby(['NMRECIBO', 'OBSERVACION'])['PTCOMISION'].sum().reset_index()
generales_ajustes4_temp['AJUSTAR?'] = np.select(
    [generales_ajustes4_temp['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["Ajustado Recibo Duplicado"],
    default=""
)
generales_ajustes4_temp = generales_ajustes4_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
generales_ajustes4 = generales_revision3.merge(generales_ajustes4_temp, on=['NMRECIBO'], how='left')

# Separación de registros que se ajustan y los que quedan para revisión
generales_ajustados4 = generales_ajustes4.loc[generales_ajustes4['AJUSTAR?'] == "Ajustado Recibo Duplicado"]
generales_revision4 = generales_ajustes4.loc[generales_ajustes4['AJUSTAR?'] != "Ajustado Recibo Duplicado"]

#### Concatenación de Pólizas para Revisión y Ajustes Realizados

In [52]:
## Generación de tabla generales nuevamente con registros ajustados y registros a revisar

generales = pd.concat([generales_revision4, generales_ajustados1, generales_ajustados2, generales_ajustados3, generales_ajustados4])
generales = generales.sort_values(by=['NMPOLIZA', 'NMRECIBO'], ascending=[True, True])
generales.reset_index(drop=True, inplace=True)

# Edición columna Observación para marcar polizas 99999999 y 999999 como revisión
generales['OBSERVACION'] = np.where((generales['AJUSTAR?'] == "") & ((generales['NMRECIBO'].fillna("") == "99999999") | (generales['NMRECIBO'].fillna("") == "999999")),"Revisar",generales['OBSERVACION'])

#### Columnas SIGNO,ABS,CANTIDAD

In [53]:
# Columna Signo y asignación de valores positivos y negativos 
generales['SIGNO'] = np.sign(generales['PTCOMISION'])
generales['POSITIVO'] = np.where(generales['SIGNO'] == 1, generales['PTCOMISION'], 0).astype(float)
generales['NEGATIVO'] = np.where(generales['SIGNO'] == -1, generales['PTCOMISION'], 0).astype(float)

# Calculo de absoluto de la comisión
generales['ABS'] = abs(generales['PTCOMISION'])

# Contar la cantidad de cada póliza/recibo
generales_cantidad = generales.groupby(['NMPOLIZA', 'NMRECIBO'])['PTCOMISION'].count().reset_index(name='CANTIDAD')
generales = generales.merge(generales_cantidad, on=['NMPOLIZA', 'NMRECIBO'], how='left')

#### Identificación de Tipo de Registro

In [54]:
# Asignarle un tipo a cada registro
generales['TIPO'] = np.select(
    [generales['CDTIPO_FUENTE'].isin(provision),
     generales['CDTIPO_FUENTE'].isin(pago)],
    ["Provision",
     "Pago"],
    default="Ajuste"
)

# Agrupar por Poliza y contar las ocurrencias de cada valor en Tipo
generales_tipo = generales[generales['TIPO'] != ""]
generales_tipo = generales_tipo.groupby('NMPOLIZA')['TIPO'].unique().reset_index(name='UNICOS')
generales_tipo['TIPO_UNICO'] = ""

for index, row in generales_tipo.iterrows():
    if len(row['UNICOS']) == 1: 
        generales_tipo.at[index, 'TIPO_UNICO'] = f"Solo {row['UNICOS'][0]}"
    else:
        generales_tipo.at[index, 'TIPO_UNICO'] = ""

generales_tipo = generales_tipo.drop('UNICOS', axis=1)

# Cruce de tabla agrupada con la completa para identificar el tipo unico
generales = generales.merge(generales_tipo, on='NMPOLIZA', how='left')

#### Pólizas en Dólares

In [55]:
## Cruce de tabla generales con base de datos de pólizas en dolares
generales =  generales.merge(dolares, left_on='NMPOLIZA', right_on='NMPOLIZA', how='left')

## Agrupar por póliza/recibo y sumar valores positivos y negativos para calcular el % pendiente
porc_pendiente = generales.groupby(['NMPOLIZA', 'NMRECIBO']).agg({'NEGATIVO': 'sum','POSITIVO':'sum','PTCOMISION':'sum'}).reset_index()
porc_pendiente['%PENDIENTE'] = np.select(
    [porc_pendiente['POSITIVO'] == 0],
    [1],
    default=porc_pendiente['PTCOMISION'] / porc_pendiente['POSITIVO']
)
porc_pendiente = porc_pendiente[['NMPOLIZA', 'NMRECIBO', '%PENDIENTE']]

## Cruce de tabla agrupada con la completa para identificar el % pendiente
generales = generales.merge(porc_pendiente, on=['NMPOLIZA', 'NMRECIBO'], how='left')

## Marcar las pólizas en dólares que tienen porcentaje pendiente y son aptas para ajuste
generales['AJUSTAR?'] = np.select(
    [(generales['DOLARES'] == "X") & (generales['%PENDIENTE'] !=0) & (generales['%PENDIENTE'].between(desde_porc_pendientes, hasta_porc_pendientes, inclusive='both'))],
    ["Ajustado Dólares"],
    default=generales['AJUSTAR?']
)

generales['OBSERVACION'] = generales['OBSERVACION'].fillna("")

## Observación para las pólizas no dólares que tienen porcentaje pendiente
for index, row in generales.iterrows():
    if (row['DOLARES'] != "X") and (row['OBSERVACION'] != "") and (row['%PENDIENTE'] !=0) and (desde_porc_pendientes <= row['%PENDIENTE'] <= hasta_porc_pendientes):
        generales.at[index, 'OBSERVACION'] = f"{row['OBSERVACION']}, Porcentaje pendiente pequeño"
    elif (row['DOLARES'] != "X") and (row['OBSERVACION'] == "") and (row['%PENDIENTE'] !=0) and (desde_porc_pendientes <= row['%PENDIENTE'] <= hasta_porc_pendientes):
        generales.at[index, 'OBSERVACION'] = f"Porcentaje pendiente pequeño"
    else:
        generales.at[index, 'OBSERVACION'] = row['OBSERVACION']

# Tabla para identificar el valor Ajustes Financieros por póliza en dólares
generales_ajustados5 = generales.loc[generales['AJUSTAR?'] == "Ajustado Dólares"]

#### Códigos Directos

In [56]:
## Cruce de tabla generales con base de datos de códigos directos
generales =  generales.merge(codigos_directos, left_on='CDAGENTE', right_on='CDAGENTE', how='left')

#### Asesores Retirados

In [57]:
## Cruce de tabla generales con base de datos de asesores retirados
generales = generales.merge(retirados, left_on='CDAGENTE', right_on='CDAGENTE', how='left')
generales['FEREGISTRO'] = pd.to_datetime(generales['FEREGISTRO'])

## Identificar si un asesor tiene registros luego de su fecha de retiro
generales['POST_RETIRO'] = np.select(
    [(generales['RETIRADOS'] == "X") & (generales['FEREGISTRO'] > generales['FEBAJA'])],
    ["Registro Post-Retiro"],
    default=""
)

## Calcular días entre la fecha de retiro del asesor y la fecha de registro de cada elemento
generales['DIAS_RETIRO'] = (generales['FEBAJA'] - generales['FEREGISTRO']).dt.days

#### Asesores FyD

In [58]:
## Cruce de tabla generales con base de datos de asesores en FyD
generales = generales.merge(asesores_formacion, left_on='CDAGENTE', right_on='CDAGENTE', how='left')

#### Comentario

In [59]:
generales['COMENTARIO'] = ""

#### Exportar Tabla con Depuraciones e Identificación de Ajustes

In [60]:
# Calcula el número total de filas en el DataFrame
total_filas = len(generales)

# Define el tamaño de la fracción
tamano_fraccion = 500000

# Calcula el número total de fracciones
total_fracciones = total_filas // tamano_fraccion + (1 if total_filas % tamano_fraccion != 0 else 0)

# Bucle para dividir y exportar en fracciones
for i in range(total_fracciones):
    inicio = i * tamano_fraccion
    fin = min((i + 1) * tamano_fraccion, total_filas)
    
    # Obtén la fracción actual del DataFrame
    fraccion_df = generales.iloc[inicio:fin]
    
    # Exporta la fracción a un archivo CSV
    fraccion_df.to_csv(f'02. Output/01. Generales/Comisiones_Generales_{i + 1}.csv', index=False, sep=';', encoding="latin1")

    print(f"Exportadas filas {inicio+1}-{fin} a fraccion_{i + 1}.csv")

Exportadas filas 1-500000 a fraccion_1.csv
Exportadas filas 500001-649567 a fraccion_2.csv


#### Generar plantillas de Ajustes Financieros

In [61]:
# Tabla para identificar el valor Ajustes Financieros por póliza
generales_ajustados1['FUENTE'] = np.select(
    [generales_ajustados1['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

generales_plantilla_ajustes1 = generales_ajustados1.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Tabla para identificar el valor Ajustes Financieros por póliza/recibo
generales_ajustados2['FUENTE'] = np.select(
    [generales_ajustados2['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

generales_plantilla_ajustes2 = generales_ajustados2.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Tabla para identificar el valor Ajustes Financieros por póliza/agente
generales_ajustados3['FUENTE'] = np.select(
    [generales_ajustados3['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

generales_plantilla_ajustes3 = generales_ajustados3.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Tabla para identificar el valor Ajustes Financieros por recibo duplicado
generales_ajustados4['FUENTE'] = np.select(
    [generales_ajustados4['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

generales_plantilla_ajustes4 = generales_ajustados4.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Concatenar dataframes de ajustes (menos por pólizas en dólares)
generales_plantilla_ajustes1_40 = pd.concat([generales_plantilla_ajustes1, generales_plantilla_ajustes2, generales_plantilla_ajustes3, generales_plantilla_ajustes4], axis=0)

In [62]:
# Parametros para clave de contabilización 40
generales_plantilla_ajustes1_40['Agrupador'] = 1
generales_plantilla_ajustes1_40['Agrupador'] = generales_plantilla_ajustes1_40['Agrupador'].cumsum()
generales_plantilla_ajustes1_40['Fecha de Documento'] = ultimo_dia_del_mes
generales_plantilla_ajustes1_40['Clase de Documento'] = "SR"
generales_plantilla_ajustes1_40['Sociedad'] = "1000"
generales_plantilla_ajustes1_40['Fecha Contabilización'] = ultimo_dia_del_mes
generales_plantilla_ajustes1_40['Período'] = ultimo_dia_del_mes[2:4]
generales_plantilla_ajustes1_40['Clave de Moneda'] = "COP"
generales_plantilla_ajustes1_40['Tipo de Cambio'] = ""
generales_plantilla_ajustes1_40['Fe.conversión '] = ""
generales_plantilla_ajustes1_40['Número documento de referencia'] = generales_plantilla_ajustes1_40['NMRECIBO']
generales_plantilla_ajustes1_40['Texto de Cebecera'] = "AJUSTE PROVISION"
generales_plantilla_ajustes1_40['Calc. Impuestos '] = ""
generales_plantilla_ajustes1_40['Clave de contabilización para la siguiente posición'] = 40
generales_plantilla_ajustes1_40['Número de identificación fiscal 1'] = ""
generales_plantilla_ajustes1_40['Tipo de número de identificación fiscal'] = ""
generales_plantilla_ajustes1_40['In. CME'] = ""
generales_plantilla_ajustes1_40['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(generales_plantilla_ajustes1_40['PTCOMISION'] > 0) & (generales_plantilla_ajustes1_40['FUENTE'] == "GW"),
     (generales_plantilla_ajustes1_40['PTCOMISION'] < 0) & (generales_plantilla_ajustes1_40['FUENTE'] == "GW"),
     (generales_plantilla_ajustes1_40['PTCOMISION'] > 0) & (generales_plantilla_ajustes1_40['FUENTE'] == "CS"),
     (generales_plantilla_ajustes1_40['PTCOMISION'] < 0) & (generales_plantilla_ajustes1_40['FUENTE'] == "CS")],
    ["2481010040",
     "5209101010",
     "2481010000",
     "5209101000"],
    default="Revisar"
)
generales_plantilla_ajustes1_40['Importe en la moneda del documento'] = abs(generales_plantilla_ajustes1_40['PTCOMISION'].round(0)).astype(int)
generales_plantilla_ajustes1_40['Indicador IVA'] = ""
generales_plantilla_ajustes1_40['Indicador retefuente WT_WITHCD'] = ""
generales_plantilla_ajustes1_40['TP retefuente WITHT'] = ""
generales_plantilla_ajustes1_40['indicador reteica WT_WITHCD'] = ""
generales_plantilla_ajustes1_40['TP reteica WITHT'] = ""
generales_plantilla_ajustes1_40['indicador reteiva WT_WITHCD'] = ""
generales_plantilla_ajustes1_40['TP reteiva WITHT'] = ""
generales_plantilla_ajustes1_40['indicador retetimbre WT_WITHCD'] = ""
generales_plantilla_ajustes1_40['Tp retetimbre WITHT'] = ""
#######################################################################
division = pd.concat([generales_ajustados1,generales_ajustados2,generales_ajustados3,generales_ajustados4], axis=0)[['NMPOLIZA','NMRECIBO','CDDELEGACION']].fillna("")
division = division.rename(columns={'CDDELEGACION':'División'})
generales_plantilla_ajustes1_40 = generales_plantilla_ajustes1_40.merge(division,on=['NMPOLIZA','NMRECIBO'], how='left')
generales_plantilla_ajustes1_40['Duplicado'] = generales_plantilla_ajustes1_40['Agrupador'].astype(str) + generales_plantilla_ajustes1_40['NMPOLIZA'] + generales_plantilla_ajustes1_40['NMRECIBO'].astype(str) + generales_plantilla_ajustes1_40['PTCOMISION'].astype(str) + generales_plantilla_ajustes1_40['Clave de contabilización para la siguiente posición'].astype(str)
generales_plantilla_ajustes1_40 = generales_plantilla_ajustes1_40.drop_duplicates(subset=['Duplicado'], keep='first')
generales_plantilla_ajustes1_40 = generales_plantilla_ajustes1_40.drop(['Duplicado'], axis=1)
generales_plantilla_ajustes1_40['División'] = np.select(
    [(generales_plantilla_ajustes1_40['División'].str.len() < 4),
     (generales_plantilla_ajustes1_40['División'] == "0000") | (generales_plantilla_ajustes1_40['División'].str.len() > 4)],
    [generales_plantilla_ajustes1_40['División'].str.rjust(4, "0"),
     "0099"],
    default=generales_plantilla_ajustes1_40['División']
)
#generales_plantilla_ajustes1_40['División'] = ""
#######################################################################
generales_plantilla_ajustes1_40['Clave de condiciones de pago'] = ""
generales_plantilla_ajustes1_40['Centro de coste'] = "10" + generales_plantilla_ajustes1_40['División'] + "000"
generales_plantilla_ajustes1_40['Centro de beneficio'] = ""
generales_plantilla_ajustes1_40['Días del descuento por pronto pago 1'] = ""
generales_plantilla_ajustes1_40['Porcentaje de descuento por pronto pago 1'] = ""
generales_plantilla_ajustes1_40['Días del descuento por pronto pago 2'] = ""
generales_plantilla_ajustes1_40['Porcentaje de descuento por pronto pago 2'] = ""
generales_plantilla_ajustes1_40['Plazo para condición de pago neto'] = ""
generales_plantilla_ajustes1_40['Fecha Contabilización'] = ultimo_dia_del_mes
generales_plantilla_ajustes1_40['Fecha base para cálculo del vencimiento'] = ""
generales_plantilla_ajustes1_40['Condición de pago fija'] = ""
generales_plantilla_ajustes1_40['Base de descuento'] = ""
generales_plantilla_ajustes1_40['Vía de pago'] = ""
generales_plantilla_ajustes1_40['Bloqueo pago'] = ""
generales_plantilla_ajustes1_40['% DPP'] = ""
generales_plantilla_ajustes1_40['Importe DPP'] = ""
generales_plantilla_ajustes1_40['Número de asignación'] = generales_plantilla_ajustes1_40['NMRECIBO']
generales_plantilla_ajustes1_40['Texto posición'] = "AJUSTE PROVISION"
generales_plantilla_ajustes1_40['Número de orden'] = ""
generales_plantilla_ajustes1_40['Clave de referencia de interlocutor comercial'] = generales_plantilla_ajustes1_40['NMPOLIZA']
generales_plantilla_ajustes1_40['Clave de referencia de interlocutor comercial2'] = generales_plantilla_ajustes1_40['CDRAMO']  
generales_plantilla_ajustes1_40['Clave de referencia para la posición de documento'] = generales_plantilla_ajustes1_40['CDAGENTE']
generales_plantilla_ajustes1_40['Tipo de banco interlocutor'] = ""
generales_plantilla_ajustes1_40['Fecha expedicion'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
generales_plantilla_ajustes1_40['Fecha fin vigencia'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
generales_plantilla_ajustes1_40['Poliza líder'] = ""
generales_plantilla_ajustes1_40['Cert.lider'] = ""
generales_plantilla_ajustes1_40['Nit'] = "8909034079"
generales_plantilla_ajustes1_40['Nombre'] = ""

# Parametros para clave de contabilización 50
generales_plantilla_ajustes1_50 = generales_plantilla_ajustes1_40.copy()
generales_plantilla_ajustes1_50['Clave de contabilización para la siguiente posición'] = 50
generales_plantilla_ajustes1_50['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(generales_plantilla_ajustes1_50['PTCOMISION'] > 0) & (generales_plantilla_ajustes1_50['FUENTE'] == "GW"),
     (generales_plantilla_ajustes1_50['PTCOMISION'] < 0) & (generales_plantilla_ajustes1_50['FUENTE'] == "GW"),
     (generales_plantilla_ajustes1_50['PTCOMISION'] > 0) & (generales_plantilla_ajustes1_50['FUENTE'] == "CS"),
     (generales_plantilla_ajustes1_50['PTCOMISION'] < 0) & (generales_plantilla_ajustes1_50['FUENTE'] == "CS")],
    ["5209101010",
     "2481010040",
     "5209101000",
     "2481010000"],
    default="Revisar"
)

# Concatenación de las dos tablas de clave de contabilización
generales_plantilla_ajustes1_final = pd.concat([generales_plantilla_ajustes1_40, generales_plantilla_ajustes1_50], axis=0)
generales_plantilla_ajustes1_final = generales_plantilla_ajustes1_final.drop(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE','PTCOMISION'], axis=1)
generales_plantilla_ajustes1_final = generales_plantilla_ajustes1_final.sort_values(by=['Agrupador', 'Clave de contabilización para la siguiente posición'])

In [63]:
# Calcula el número total de filas en el DataFrame
total_filas2 = len(generales_plantilla_ajustes1_final)

# Define el tamaño de la fracción
tamano_fraccion2 = 19998

# Calcula el número total de fracciones
total_fracciones2 = total_filas2 // tamano_fraccion2 + (1 if total_filas2 % tamano_fraccion2 != 0 else 0)

# Bucle para dividir y exportar en fracciones
for i in range(total_fracciones2):
    inicio2 = i * tamano_fraccion2
    fin2 = min((i + 1) * tamano_fraccion2, total_filas2)
    
    # Obtén la fracción actual del DataFrame
    fraccion_df2 = generales_plantilla_ajustes1_final.iloc[inicio2:fin2]
    
    # Exporta la fracción a un archivo CSV
    fraccion_df2.to_csv(f'02. Output/01. Generales/Ajustes/Ajustes Financieros {i + 1}.csv', index=False, sep=';', encoding="latin1")

    print(f"Exportadas filas {inicio2+1}-{fin2} a fraccion_{i + 1}.csv")

Exportadas filas 1-19998 a fraccion_1.csv
Exportadas filas 19999-39996 a fraccion_2.csv
Exportadas filas 39997-59994 a fraccion_3.csv
Exportadas filas 59995-79992 a fraccion_4.csv
Exportadas filas 79993-99990 a fraccion_5.csv
Exportadas filas 99991-119988 a fraccion_6.csv
Exportadas filas 119989-139986 a fraccion_7.csv
Exportadas filas 139987-159984 a fraccion_8.csv
Exportadas filas 159985-179982 a fraccion_9.csv
Exportadas filas 179983-199980 a fraccion_10.csv
Exportadas filas 199981-219978 a fraccion_11.csv
Exportadas filas 219979-239976 a fraccion_12.csv
Exportadas filas 239977-259974 a fraccion_13.csv
Exportadas filas 259975-279972 a fraccion_14.csv
Exportadas filas 279973-292136 a fraccion_15.csv


In [64]:
# Tabla para identificar el valor Ajustes Financieros por póliza en dólares
generales_ajustados5['FUENTE'] = np.select(
    [generales_ajustados5['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

generales_plantilla_ajustes2_40 = generales_ajustados5.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

In [67]:
# Parametros para clave de contabilización 40
generales_plantilla_ajustes2_40['Agrupador'] = 1
generales_plantilla_ajustes2_40['Agrupador'] = generales_plantilla_ajustes2_40['Agrupador'].cumsum()
generales_plantilla_ajustes2_40['Fecha de Documento'] = ultimo_dia_del_mes
generales_plantilla_ajustes2_40['Clase de Documento'] = "SR"
generales_plantilla_ajustes2_40['Sociedad'] = "1000"
generales_plantilla_ajustes2_40['Fecha Contabilización'] = ultimo_dia_del_mes
generales_plantilla_ajustes2_40['Período'] = ultimo_dia_del_mes[2:4]
generales_plantilla_ajustes2_40['Clave de Moneda'] = "USD"
generales_plantilla_ajustes2_40['Tipo de Cambio'] = ""
generales_plantilla_ajustes2_40['Fe.conversión '] = ""
generales_plantilla_ajustes2_40['Número documento de referencia'] = generales_plantilla_ajustes2_40['NMRECIBO']
generales_plantilla_ajustes2_40['Texto de Cebecera'] = "MONEDA"
generales_plantilla_ajustes2_40['Calc. Impuestos '] = ""
generales_plantilla_ajustes2_40['Clave de contabilización para la siguiente posición'] = 40
generales_plantilla_ajustes2_40['Número de identificación fiscal 1'] = ""
generales_plantilla_ajustes2_40['Tipo de número de identificación fiscal'] = ""
generales_plantilla_ajustes2_40['In. CME'] = ""
generales_plantilla_ajustes2_40['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(generales_plantilla_ajustes2_40['PTCOMISION'] > 0) & (generales_plantilla_ajustes2_40['FUENTE'] == "GW"),
     (generales_plantilla_ajustes2_40['PTCOMISION'] < 0) & (generales_plantilla_ajustes2_40['FUENTE'] == "GW"),
     (generales_plantilla_ajustes2_40['PTCOMISION'] > 0) & (generales_plantilla_ajustes2_40['FUENTE'] == "CS"),
     (generales_plantilla_ajustes2_40['PTCOMISION'] < 0) & (generales_plantilla_ajustes2_40['FUENTE'] == "CS")],
    ["2481010040",
     "5209101010",
     "2481010000",
     "5209101000"],
    default="Revisar"
)
generales_plantilla_ajustes2_40['Importe en la moneda del documento'] = abs(generales_plantilla_ajustes2_40['PTCOMISION'].round(0)).astype(int)
generales_plantilla_ajustes2_40['Indicador IVA'] = ""
generales_plantilla_ajustes2_40['Indicador retefuente WT_WITHCD'] = ""
generales_plantilla_ajustes2_40['TP retefuente WITHT'] = ""
generales_plantilla_ajustes2_40['indicador reteica WT_WITHCD'] = ""
generales_plantilla_ajustes2_40['TP reteica WITHT'] = ""
generales_plantilla_ajustes2_40['indicador reteiva WT_WITHCD'] = ""
generales_plantilla_ajustes2_40['TP reteiva WITHT'] = ""
generales_plantilla_ajustes2_40['indicador retetimbre WT_WITHCD'] = ""
generales_plantilla_ajustes2_40['Tp retetimbre WITHT'] = ""
#######################################################################
division2 = generales_ajustados5[['NMPOLIZA','NMRECIBO','CDDELEGACION']].fillna("")
division2 = division2.rename(columns={'CDDELEGACION':'División'})
generales_plantilla_ajustes2_40 = generales_plantilla_ajustes2_40.merge(division2,on=['NMPOLIZA','NMRECIBO'], how='left')
generales_plantilla_ajustes2_40['Duplicado'] = generales_plantilla_ajustes2_40['Agrupador'].astype(str) + generales_plantilla_ajustes2_40['NMPOLIZA'] + generales_plantilla_ajustes2_40['NMRECIBO'].astype(str) + generales_plantilla_ajustes2_40['PTCOMISION'].astype(str) + generales_plantilla_ajustes2_40['Clave de contabilización para la siguiente posición'].astype(str)
generales_plantilla_ajustes2_40 = generales_plantilla_ajustes2_40.drop_duplicates(subset=['Duplicado'], keep='first')
generales_plantilla_ajustes2_40 = generales_plantilla_ajustes2_40.drop(['Duplicado'], axis=1)
generales_plantilla_ajustes2_40['División'] = np.select(
    [(generales_plantilla_ajustes2_40['División'].str.len() < 4),
     (generales_plantilla_ajustes2_40['División'] == "0000") | (generales_plantilla_ajustes2_40['División'].str.len() > 4)],
    [generales_plantilla_ajustes2_40['División'].str.rjust(4, "0"),
     "0099"],
    default=generales_plantilla_ajustes2_40['División']
)
#generales_plantilla_ajustes1_40['División'] = ""
#######################################################################
generales_plantilla_ajustes2_40['Clave de condiciones de pago'] = ""
generales_plantilla_ajustes2_40['Centro de coste'] = "10" + generales_plantilla_ajustes2_40['División'] + "000"
generales_plantilla_ajustes2_40['Centro de beneficio'] = ""
generales_plantilla_ajustes2_40['Días del descuento por pronto pago 1'] = ""
generales_plantilla_ajustes2_40['Porcentaje de descuento por pronto pago 1'] = ""
generales_plantilla_ajustes2_40['Días del descuento por pronto pago 2'] = ""
generales_plantilla_ajustes2_40['Porcentaje de descuento por pronto pago 2'] = ""
generales_plantilla_ajustes2_40['Plazo para condición de pago neto'] = ""
generales_plantilla_ajustes2_40['Fecha Contabilización'] = ultimo_dia_del_mes
generales_plantilla_ajustes2_40['Fecha base para cálculo del vencimiento'] = ""
generales_plantilla_ajustes2_40['Condición de pago fija'] = ""
generales_plantilla_ajustes2_40['Base de descuento'] = ""
generales_plantilla_ajustes2_40['Vía de pago'] = ""
generales_plantilla_ajustes2_40['Bloqueo pago'] = ""
generales_plantilla_ajustes2_40['% DPP'] = ""
generales_plantilla_ajustes2_40['Importe DPP'] = ""
generales_plantilla_ajustes2_40['Número de asignación'] = generales_plantilla_ajustes2_40['NMRECIBO']
generales_plantilla_ajustes2_40['Texto posición'] = "AJUSTE PROVISION"
generales_plantilla_ajustes2_40['Número de orden'] = ""
generales_plantilla_ajustes2_40['Clave de referencia de interlocutor comercial'] = generales_plantilla_ajustes2_40['NMPOLIZA']
generales_plantilla_ajustes2_40['Clave de referencia de interlocutor comercial2'] = generales_plantilla_ajustes2_40['CDRAMO']  
generales_plantilla_ajustes2_40['Clave de referencia para la posición de documento'] = generales_plantilla_ajustes2_40['CDAGENTE']
generales_plantilla_ajustes2_40['Tipo de banco interlocutor'] = ""
generales_plantilla_ajustes2_40['Fecha expedicion'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
generales_plantilla_ajustes2_40['Fecha fin vigencia'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
generales_plantilla_ajustes2_40['Poliza líder'] = ""
generales_plantilla_ajustes2_40['Cert.lider'] = ""
generales_plantilla_ajustes2_40['Nit'] = "8909034079"
generales_plantilla_ajustes2_40['Nombre'] = ""

# Parametros para clave de contabilización 50
generales_plantilla_ajustes2_50 = generales_plantilla_ajustes2_40.copy()
generales_plantilla_ajustes2_50['Clave de contabilización para la siguiente posición'] = 50
generales_plantilla_ajustes2_50['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(generales_plantilla_ajustes2_50['PTCOMISION'] > 0) & (generales_plantilla_ajustes2_50['FUENTE'] == "GW"),
     (generales_plantilla_ajustes2_50['PTCOMISION'] < 0) & (generales_plantilla_ajustes2_50['FUENTE'] == "GW"),
     (generales_plantilla_ajustes2_50['PTCOMISION'] > 0) & (generales_plantilla_ajustes2_50['FUENTE'] == "CS"),
     (generales_plantilla_ajustes2_50['PTCOMISION'] < 0) & (generales_plantilla_ajustes2_50['FUENTE'] == "CS")],
    ["5209101010",
     "2481010040",
     "5209101000",
     "2481010000"],
    default="Revisar"
)

# Concatenación de las dos tablas de clave de contabilización
generales_plantilla_ajustes2_final = pd.concat([generales_plantilla_ajustes2_40, generales_plantilla_ajustes2_50], axis=0)
generales_plantilla_ajustes2_final = generales_plantilla_ajustes2_final.drop(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE','PTCOMISION'], axis=1)
generales_plantilla_ajustes2_final = generales_plantilla_ajustes2_final.sort_values(by=['Agrupador', 'Clave de contabilización para la siguiente posición'])

In [69]:
# Calcula el número total de filas en el DataFrame
total_filas3 = len(generales_plantilla_ajustes2_final)

# Define el tamaño de la fracción
tamano_fraccion3 = 19998

# Calcula el número total de fracciones
total_fracciones3 = total_filas3 // tamano_fraccion3 + (1 if total_filas3 % tamano_fraccion3 != 0 else 0)

# Bucle para dividir y exportar en fracciones
for i in range(total_fracciones3):
    inicio3 = i * tamano_fraccion3
    fin3 = min((i + 1) * tamano_fraccion3, total_filas3)
    
    # Obtén la fracción actual del DataFrame
    fraccion_df3 = generales_plantilla_ajustes2_final.iloc[inicio3:fin3]
    
    # Exporta la fracción a un archivo CSV
    fraccion_df3.to_csv(f'02. Output/01. Generales/Ajustes Dólares/Ajustes Financieros {i + 1}.csv', index=False, sep=';', encoding="latin1")

    print(f"Exportadas filas {inicio3+1}-{fin3} a fraccion_{i + 1}.csv")

Exportadas filas 1-5100 a fraccion_1.csv
